# Pre-processing Nevada Allocation data for WaDEQA upload.
Date Updated: 08/03/2020
Purpose:  To pre-process the Nevada data into one master file for simple DataFrame creation and extraction

### Notes:
To incldue owners, made a temporary Permit_Owners_5temp.csv file by removing previous onwers from list.  Plan on left-join to POD AllApps_2.csv by 'app' field.


https://ndwr.maps.arcgis.com/home/item.html?id=0d050f7b79724404b80bf29589f67363
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/WaterRights_POD_POU/FeatureServer
https://arcgis.shpo.nv.gov/arcgis/rest/services/Water_Resources_Public_Data/SE_HydrographicBasins/FeatureServer
http://water.nv.gov/CodeDefinitions.aspx

In [ ]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [ ]:
#Working Directory
workingDir = "G:/Shared drives/WaDE Data/Nevada/WaterAllocation/RawInputData"
os.chdir(workingDir)

## POD Sites Data

In [ ]:
# POD Data
PoDAAInput = "POD AllApps_2_input.csv"
dfPoD = pd.read_csv(PoDAAInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPoD:
    dfPoD['WaDEUUID'] = "nvD" + dfPoD.index.astype(str)
    dfPoD.to_csv('POD AllApps_2_input.csv', index=False)
    
dfPoD['in_PODorPOUSite'] = "POD"
dfPoD['in_SiteNativeID'] = "POD" + dfPoD.index.astype(str) # creating custom site Native iD for POD

print(len(dfPoD))
dfPoD.head(1)

## PoU Sites Data

In [ ]:
# PoU Data
PoUAAInput = "PoU AllApps_3_input.csv"
dfPoU = pd.read_csv(PoUAAInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfPoU:
    dfPoU['WaDEUUID'] = "nvU" + dfPoU.index.astype(str)
    dfPoU.to_csv('PoU AllApps_3_input.csv.csv', index=False)
    
dfPoU['in_PODorPOUSite'] = "POU"
dfPoU['in_SiteNativeID'] = "POU" + dfPoU.index.astype(str) # creating custom site Native iD for POU

print(len(dfPoU))
dfPoU.head(1)

## Concatenate POD & PoU

In [ ]:
# concat POD and POU data
df = pd.concat([dfPoD, dfPoU], ignore_index=True)
df = df.drop_duplicates().reset_index()
print(len(df))
df.head(1)

## Owner Data
- add to concatenate

In [ ]:
# Owner Data
OwnTemp = "Permit_Owners_5temp.csv"
dfown = pd.read_csv(OwnTemp)
print(len(dfown))
dfown.head(1)

In [ ]:
# Clean Owner info. Remove special characters
import re

def cleanOwnerDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).strip()
    return Val

dfown['owner_name'] = dfown.apply(lambda row: cleanOwnerDataFunc(row['owner_name']), axis=1)
dfown['owner_name'].unique()

In [ ]:
# With owner sort and merge columns by 'app' field.
dfown = dfown.groupby('app', sort=False).agg(lambda x: ', '.join([str(elem) for elem in (list(set(x)))]))
dfown = dfown.drop_duplicates().reset_index()
print(len(dfown))
dfown.head(1)

In [ ]:
#Merge data with Owner by app field
df = pd.merge(df, dfown, left_on='app', right_on='app', how='left') # Joinning PoD data
df.head()

## Clean Data

In [ ]:
# For creating WaterSourceTypeCV
UnknownWSCVDict = {
"EFF" : "Reuse",
"GEO" : "Groundwater",
"LAK" : "Surface Water",
"OGW" : "Groundwater",
"OSW" : "Surface Water",
"RES" : "Reservoir",
"SPR" : "Surface Water",
"STO" : "Storage",
"STR" : "Surface Water",
"UG" : "Groundwater",
"UKN" : "Unknown"}
def assignWaterSourceTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = UnknownWSCVDict[String1]
        except:
            outList = "Unspecified"
    return outList

df['in_WaterSourceTypeCV'] = df.apply(lambda row: assignWaterSourceTypeCV(row['source']), axis=1)

In [ ]:
# For creating SiteName
def assignSiteName(colrowValue):
    colrowValuestr = str(colrowValue)
    colrowValuestr = colrowValuestr.strip()
    if ((colrowValuestr == "") or pd.isnull(colrowValuestr) or colrowValuestr == 'nan'):
        outList = "Unspecified"
    else:
        outList = colrowValuestr
    return outList

df['in_SiteName'] = df.apply(lambda row: assignSiteName(row['site_name']), axis=1)
df.head(1)

In [ ]:
# For creating SiteTypeCV
UnknownSTCVDict = {
    "EFF":"Effluent",
    "GEO":"Geothermal",
    "LAK":"lake",
    "OGW":"Other Ground Water",
    "OSW":"Other Surface Water",
    "RES":"Reservoir",
    "SPR":"Spring",
    "STO":"Storage",
    "STR":"stream",
    "UG":"Underground",
    "UKN":"Unknown"}
def assignSiteTypeCV(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = UnknownSTCVDict[String1]
        except:
            outList = "Unspecified"
    return outList

df['in_SiteTypeCV'] = df.apply(lambda row: assignSiteTypeCV(row['source']), axis=1)
df.head(1)

## WaDE Custom Elements (due to missing state site info)

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDENV_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = df['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A):
    if (A == '') or (pd.isnull(A)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceTypeCV'] == A), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

df['in_WaterSourceNativeID'] = df.apply(lambda row: retrieveWaterSourceNativeID(row['in_WaterSourceTypeCV']), axis=1)
df.head(1)

In [ ]:
#Changing datatype of used date fields. 
df['prior_dt'] = pd.to_datetime(df['prior_dt'], errors = 'coerce')
df['prior_dt'] = pd.to_datetime(df["prior_dt"].dt.strftime('%m/%d/%Y'))
df.head(1)

## Shapefile Data
- For attaching gemetry to csv inputs.

In [ ]:
# PoU Shapefile Data, Shapefile input
dfPoUshapetemp = gpd.read_file('shapefile/NVwr_POU.shp')
#dfPoUshapetemp = pd.DataFrame(dfPoUshapetemp)
dfPoUshapetemp.head(3)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp.index.astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

### Exporting to Finished File

In [ ]:
# Output
df.to_csv('P_MastersNV.csv', index=False)
dfPoUshape.to_csv('P_nvGeometry.csv', index=False) # The output geometry.